# Hybrid Retrieval: Combining BM25 and Vector Search

- **BM25**: Excels at finding exact keyword matches but misses semantic relationships
- **Vector Search**: Captures semantic meaning but may miss exact term matches

## Key Hybrid Retrieval Concepts

### 1. Fusion Approaches
Fusion approaches retrieve documents separately with each method, then combine the results.

- **Simple fusion**: Taking the union of results from multiple retrievers
- **Weighted fusion**: Adjusting scores from each retriever and combining them
- **Reciprocal rank fusion**: Considering the rank position of documents in each result set

### 2. Ensemble Approaches
Ensemble approaches use multiple retrievers in series or with more complex logic.

- **Sequential retrieval**: Using one retriever's output as input to another
- **Filter-then-rank**: Using one method to create a candidate pool, another to rank

In this notebook, we'll implement and evaluate different hybrid retrieval strategies.

## Setup and Dependencies
First, let's install the necessary dependencies.

In [1]:
# This installs the embeddings, it can take a while to install so I didn't include in the the default requirements.txt
!uv pip install llama-index-embeddings-huggingface

Using Python 3.12.10 environment at: /workspaces/fundamentals-of-ai-engineering-principles-and-practical-applications-6026542/.venv
Audited 1 package in 11ms


## Setup and Imports

First, let's import the necessary libraries:

In [2]:
# Imports
import pandas as pd
from llama_index.core import Document, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.schema import QueryBundle, NodeWithScore
from llama_index.core.retrievers import BaseRetriever
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

## Creating Sample Documents

Let's create a collection of AI-related documents to test our retrieval methods:

In [3]:
# Create sample AI-related documents
documents = [
    Document(text="Machine learning is a subset of artificial intelligence that involves building systems that can learn from data. Common machine learning algorithms include linear regression, decision trees, and neural networks."),
    Document(text="BM25, or Best Match 25, also known as Okapi BM25, is a ranking algorithm for information retrieval and search engines that determines a document's relevance to a given query and ranks documents based on their relevance scores."),
    Document(text="Neural networks are computational models inspired by the human brain. They consist of layers of interconnected nodes or 'neurons' that process and transform input data to produce meaningful outputs."),
    Document(text="Transformers are a type of deep learning architecture introduced in the paper 'Attention is All You Need'. They have revolutionized natural language processing tasks such as translation, summarization, and question answering."),
    Document(text="Backpropagation is a key algorithm for training neural networks. It calculates the gradient of the loss function with respect to the network weights, allowing for efficient optimization."),
    Document(text="Computer vision is an interdisciplinary field that deals with how computers can gain high-level understanding from digital images or videos. It aims to automate tasks that the human visual system can do."),
    Document(text="Reinforcement learning is a type of machine learning where an agent learns to make decisions by taking actions in an environment to maximize a reward signal. It has been used to achieve superhuman performance in games like chess, Go, and various Atari games."),
    Document(text="Large Language Models (LLMs) like GPT-4 and Claude are transformer-based models trained on vast amounts of text data. They can generate human-like text, answer questions, and perform a variety of language tasks."),
    Document(text="Natural Language Processing (NLP) encompasses techniques for understanding, interpreting and generating human language. Modern NLP systems use transformer architectures to process and generate text with remarkable accuracy."),
    Document(text="Deep learning is a subset of machine learning that uses multi-layered neural networks to learn from data. It has achieved breakthroughs in computer vision, speech recognition, and natural language processing."),
]

## Building Individual Retrievers

Now, let's create both BM25 and Vector retrievers that we'll use as components of our hybrid approach:

In [4]:
# Create the individual BM25 and Vector retrievers
def create_retrievers(documents, top_k=5):
    """Create BM25 and Vector retrievers from documents.
    
    Args:
        documents: List of Document objects
        top_k: Number of results to return
        
    Returns:
        Dictionary containing BM25 and Vector retrievers
    """
    # First, parse documents into nodes (chunks)
    parser = SentenceSplitter(chunk_size=2000, chunk_overlap=0)
    nodes = parser.get_nodes_from_documents(documents)
    
    # Create BM25 retriever (lexical search)
    bm25_retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=top_k)
    
    # Create Vector retriever (semantic search)
    # First load the embedding model
    embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_index = VectorStoreIndex(nodes, embed_model=embed_model)
    vector_retriever = vector_index.as_retriever(similarity_top_k=top_k)

    # Return both retrievers in a dictionary
    return {
        "BM25": bm25_retriever,
        "Vector": vector_retriever,
        "nodes": nodes  # Also return nodes for reference
    }

## Simple Fusion Retriever

A simple fusion retriever that combines the results from both BM25 and vector search without any complex re-ranking. This approach:

1. Runs both retrievers independently
2. Takes the union of their results
3. Removes duplicate documents

This is the most straightforward hybrid approach but can be very effective.

In [5]:
# Create a simple hybrid retriever
class SimpleFusionRetriever(BaseRetriever):
    """Simple fusion retriever that combines results from multiple retrievers.
    
    This retriever runs multiple retrieval methods independently and takes the union
    of their results, removing duplicates.
    """

    def __init__(
        self,
        retrievers: dict,
    ):
        """Initialize with retrievers dictionary."""
        self.retrievers = retrievers
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> list[NodeWithScore]:
        """Retrieve nodes given query."""
        # Dictionary to store unique nodes by ID to remove duplicates
        all_nodes = {}

        # Run each retriever and collect results
        for name, retriever in self.retrievers.items():
            results = retriever.retrieve(query_bundle)

            # Add to results dictionary (keyed by node_id for deduplication)
            for node in results:
                all_nodes[node.node.node_id] = node
                
        # Return all unique nodes
        return list(all_nodes.values())

## Testing Our Retrievers

Now, let's create instances of each retriever and see how they compare on a sample query. We'll format the results to make them easier to read and analyze.

In [6]:
# Create individual retrievers
retrievers_dict = create_retrievers(documents, top_k=5)

# Create simple fusion retriever
hybrid_retriever = SimpleFusionRetriever(
    retrievers={k: v for k, v in retrievers_dict.items() if k != "nodes"}
)

# Format results helper function
def format_results(results, name):
    """Format retrieval results for display in a clean, readable format."""
    lines = [f"\n{name}:"]

    for i, node in enumerate(results):
        # Include result number, score, and first 130 chars of text on one line
        text = node.node.text
        summary = text

        # Format with clear separation between result number and content
        lines.append(f"{i+1}. [Score: {node.score:.4f}] {summary}")

    return "\n".join(lines)

# Test the retrievers
query_text = "algorithm"
query_bundle = QueryBundle(query_text)

print(f"Query: {query_text}")
print("=" * 30)

# Get results from each retriever
bm25_results = retrievers_dict["BM25"].retrieve(query_bundle)
vector_results = retrievers_dict["Vector"].retrieve(query_bundle)
hybrid_results = hybrid_retriever.retrieve(query_bundle)

# Print results
print(format_results(bm25_results, "BM25 Retriever"))
print(format_results(vector_results, "Vector Retriever"))
print(format_results(hybrid_results, "Simple Fusion Retriever"))

Query: algorithm

BM25 Retriever:
1. [Score: 0.5282] Backpropagation is a key algorithm for training neural networks. It calculates the gradient of the loss function with respect to the network weights, allowing for efficient optimization.
2. [Score: 0.4553] Machine learning is a subset of artificial intelligence that involves building systems that can learn from data. Common machine learning algorithms include linear regression, decision trees, and neural networks.
3. [Score: 0.4465] BM25, or Best Match 25, also known as Okapi BM25, is a ranking algorithm for information retrieval and search engines that determines a document's relevance to a given query and ranks documents based on their relevance scores.
4. [Score: 0.0000] Deep learning is a subset of machine learning that uses multi-layered neural networks to learn from data. It has achieved breakthroughs in computer vision, speech recognition, and natural language processing.
5. [Score: 0.0000] Natural Language Processing (NLP) en

### Analysis of Simple Fusion Results

Looking at the results above, we can see:

1. **BM25 Retriever**: Found documents with the exact term "algorithm" with high scores.
2. **Vector Retriever**: Found semantically related documents, even those that may not contain the exact term.
3. **Simple Fusion**: Combined the results from both retrievers, giving us a more comprehensive set of documents.

However, we can see a limitation: the simple fusion approach doesn't rerank or adjust scores, so the quality of the combined results depends entirely on the individual retrievers' scores, which aren't directly comparable.

This brings us to our next approach: weighted fusion.

## Weighted Fusion Retriever

The weighted fusion approach improves upon simple fusion by:

1. Allowing us to assign different weights to each retriever
2. Combining scores in a way that preserves their relative importance
3. Reranking the final results based on the combined scores

This gives us more control over the balance between lexical and semantic search.

In [7]:
# Create a more advanced hybrid retriever with weighted scoring
class WeightedFusionRetriever(BaseRetriever):
    """Weighted fusion retriever that combines and rescores results.
    
    This retriever assigns weights to different retrieval methods, then combines 
    their scores and reranks the results based on the combined score.
    """

    def __init__(
        self,
        retrievers: dict,
        weights: dict
    ):
        """Initialize with retrievers and weights.
        
        Args:
            retrievers: Dictionary of retrievers (name -> retriever)
            weights: Dictionary of weights for each retriever (name -> weight)
        """
        self.retrievers = retrievers
        self.weights = weights
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> list[NodeWithScore]:
        """Retrieve nodes with weighted fusion approach."""
        # Get results from each retriever
        all_results = {}

        for name, retriever in self.retrievers.items():
            results = retriever.retrieve(query_bundle)
            weight = self.weights.get(name, 1.0)  # Default weight 1.0

            for node_with_score in results:
                node_id = node_with_score.node.node_id

                # Apply weight to score
                weighted_score = node_with_score.score * weight

                # Store the node and its scores
                if node_id not in all_results:
                    all_results[node_id] = {
                        "node": node_with_score.node,
                        "scores": {}
                    }

                all_results[node_id]["scores"][name] = weighted_score

        # Combine scores and create final results
        final_results = []
        for node_id, data in all_results.items():
            # Sum up the scores from different retrievers
            combined_score = sum(data["scores"].values())

            # Create NodeWithScore object
            node_with_score = NodeWithScore(
                node=data["node"],
                score=combined_score
            )
            final_results.append(node_with_score)

        # Sort by combined score (descending)
        final_results.sort(key=lambda x: x.score, reverse=True)

        return final_results

## Testing Weighted Fusion with Different Weight Configurations

Now let's create three versions of our weighted fusion retriever with different weights:

1. **BM25-Heavy**: Favors lexical matching (80% BM25, 20% Vector)
2. **Vector-Heavy**: Favors semantic understanding (20% BM25, 80% Vector)
3. **Balanced**: Equal weights (50% BM25, 50% Vector)

We'll test them on a more semantic query to see how the weights affect the results.

In [8]:
# Test the weighted fusion retriever with different weights
# Create weighted fusion retrievers with different weight configurations
bm25_heavy = WeightedFusionRetriever(
    retrievers={k: v for k, v in retrievers_dict.items() if k != "nodes"},
    weights={"BM25": 0.8, "Vector": 0.2}  # BM25 heavy
)

vector_heavy = WeightedFusionRetriever(
    retrievers={k: v for k, v in retrievers_dict.items() if k != "nodes"},
    weights={"BM25": 0.2, "Vector": 0.8}  # Vector heavy
)

balanced = WeightedFusionRetriever(
    retrievers={k: v for k, v in retrievers_dict.items() if k != "nodes"},
    weights={"BM25": 0.5, "Vector": 0.5}  # Balanced weights
)

# Test with a semantic query
semantic_query = "how do computers understand images?"
semantic_bundle = QueryBundle(semantic_query)

print(f"\n\nQuery: {semantic_query}")
print("=" * 80)

# Get results from weighted retrievers
bm25_heavy_results = bm25_heavy.retrieve(semantic_bundle)
vector_heavy_results = vector_heavy.retrieve(semantic_bundle)
balanced_results = balanced.retrieve(semantic_bundle)

# Print results
print(format_results(bm25_heavy_results, "BM25-Heavy Weighted Fusion"))
print(format_results(vector_heavy_results, "Vector-Heavy Weighted Fusion"))
print(format_results(balanced_results, "Balanced Weighted Fusion"))



Query: how do computers understand images?

BM25-Heavy Weighted Fusion:
1. [Score: 2.9622] Computer vision is an interdisciplinary field that deals with how computers can gain high-level understanding from digital images or videos. It aims to automate tasks that the human visual system can do.
2. [Score: 0.5407] Natural Language Processing (NLP) encompasses techniques for understanding, interpreting and generating human language. Modern NLP systems use transformer architectures to process and generate text with remarkable accuracy.
3. [Score: 0.4552] Neural networks are computational models inspired by the human brain. They consist of layers of interconnected nodes or 'neurons' that process and transform input data to produce meaningful outputs.
4. [Score: 0.4291] Deep learning is a subset of machine learning that uses multi-layered neural networks to learn from data. It has achieved breakthroughs in computer vision, speech recognition, and natural language processing.
5. [Score: 0.0

### Analysis of Weighted Fusion Results

In this example, we used a more natural language query: "how do computers understand images?"

Observations:
1. All three approaches correctly identified the computer vision document as most relevant.
2. The **BM25-Heavy** approach gave a much higher score (2.96) to the top document compared to the **Vector-Heavy** approach (1.20).
3. The **Balanced** approach shows a good compromise (2.08) between the two.
4. The ranking of secondary documents (2-5) is slightly different across the approaches.

This demonstrates how adjusting weights lets us control the balance between exact keyword matching and semantic understanding in our retrieval system. The best configuration depends on the types of queries your application typically receives.

## Comprehensive Comparison Across Different Query Types

Now, let's compare all our retrievers across a variety of query types to better understand their strengths and weaknesses:

In [9]:
# Compare retriever performance across different query types
# Function to compare retrievers
def compare_retrievers(retrievers, queries):
    """Compare multiple retrievers across different query types with clearer output.
    
    Args:
        retrievers: Dictionary of retrievers to compare
        queries: List of query strings to test
        
    Returns:
        Formatted string with comparison results
    """
    # Set up the results container
    results = []

    # Process each query
    for query in queries:
        query_bundle = QueryBundle(query)
        query_results = {}

        # For each retriever, get top results
        for name, retriever in retrievers.items():
            retrieved = retriever.retrieve(query_bundle)
            # Get first sentence of each result for compact display
            top_results = []
            # Limit to top 3 for clarity
            for i, node in enumerate(retrieved[:3]):
                first_sentence = node.node.text.split('.')[0].strip()
                # Truncate if too long
                if len(first_sentence) > 50:
                    first_sentence = first_sentence[:47] + "..."
                top_results.append(f"{i+1}. {first_sentence}")

            query_results[name] = top_results

        results.append((query, query_results))

    # Format the output as readable text
    output = ["RETRIEVER COMPARISON ACROSS QUERY TYPES", "=" * 80, ""]

    # For each query
    for query, query_results in results:
        output.append(f"QUERY: \"{query}\"")
        output.append("-" * 80)

        # For each retriever
        for retriever_name, results_list in query_results.items():
            output.append(f"\n{retriever_name} Results:")
            if not results_list:
                output.append("  No results found")
            else:
                for result in results_list:
                    output.append(f"  {result}")

        output.append("\n" + "=" * 80)

    return "\n".join(output)


all_retrievers = {
    "BM25": retrievers_dict["BM25"],
    "Vector": retrievers_dict["Vector"],
    "Fusion": hybrid_retriever,
    "BM25-Heavy": bm25_heavy,
    "Vector-Heavy": vector_heavy
}

# Test with a variety of queries
test_queries = [
    "CV libs",                           # Abbreviation and technical term
    "neural networks backpropagation",   # Technical keyword search
    "how do computers understand images?",  # Natural language question
    "natural language processing",       # Direct field search
    "machine learning applications",     # General topic search
    "reinforcement learning in games"    # Specific application search
]

# Usage remains the same
comparison_text = compare_retrievers(all_retrievers, test_queries)
print(comparison_text)

RETRIEVER COMPARISON ACROSS QUERY TYPES

QUERY: "CV libs"
--------------------------------------------------------------------------------

BM25 Results:
  1. Deep learning is a subset of machine learning t...
  2. Natural Language Processing (NLP) encompasses t...
  3. Large Language Models (LLMs) like GPT-4 and Cla...

Vector Results:
  1. Large Language Models (LLMs) like GPT-4 and Cla...
  2. Transformers are a type of deep learning archit...
  3. Deep learning is a subset of machine learning t...

Fusion Results:
  1. Deep learning is a subset of machine learning t...
  2. Natural Language Processing (NLP) encompasses t...
  3. Large Language Models (LLMs) like GPT-4 and Cla...

BM25-Heavy Results:
  1. Large Language Models (LLMs) like GPT-4 and Cla...
  2. Transformers are a type of deep learning archit...
  3. Deep learning is a subset of machine learning t...

Vector-Heavy Results:
  1. Large Language Models (LLMs) like GPT-4 and Cla...
  2. Transformers are a type of deep lea

## Key Findings and Practical Tips

Based on our experiments, we can draw several conclusions and offer practical advice for implementing hybrid retrievers in real-world applications:

# Practical tips for implementing hybrid retrievers:

1. **Choosing weights**: Start with equal weights and adjust based on the query types you see in your application. If users frequently use keywords, increase BM25 weight; if they ask natural language questions, favor vector search.

2. **Performance considerations**: The fusion approach requires running both retrievers, which can be more expensive. In production, consider:

    - Running retrievers in parallel
    - Using a faster first-stage retriever (typically BM25) to filter candidates
    - Caching results for common queries

3. **Beyond simple weighting**: More sophisticated approaches include:

    - Using machine learning to learn optimal weights
    - Adjusting weights dynamically based on query type
    - Implementing reciprocal rank fusion, which considers result positions

4. **Evaluation**: Always evaluate your hybrid approach on a diverse set of queries to ensure it's performing better than either method alone.

## Conclusion

In this notebook, we've explored different approaches to hybrid retrieval, combining the strengths of BM25 lexical search and vector-based semantic search.

Key takeaways:
- Simple fusion provides an easy way to combine results but doesn't rerank them
- Weighted fusion offers more control over the balance between approaches
- The best configuration depends on your specific use case and query patterns
- Hybrid approaches generally outperform either method alone across diverse query types

By implementing hybrid retrieval, you can create more robust information retrieval systems that work well for both keyword searches and natural language queries.